In [1]:
from read_edf import plot_signals
from read_result import read_result_file
from read_result import plot_sleep_stages
from split_signal import split_edf_by_annotations
from split_signal import split_edf_by_30s
from sklearn.decomposition import PCA
import processing_data
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm

In [2]:
feature_number = 105

In [3]:
X = np.empty((0,feature_number))
y = np.empty((0,))
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X = np.concatenate((X, file['features']))
        y = np.concatenate((y, file['labels']))
print(X.shape)
print(y.shape)

(393345, 105)
(393345,)


In [4]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.fit_transform(X)

In [5]:
pca = PCA(n_components= 0.999).fit(X_scaled)
X_pca = pca.transform(X_scaled)
print(X_pca.shape)

(393345, 82)


In [6]:
model_X = X_pca
model_y = y
print(model_X.shape, model_y.shape)

(393345, 82) (393345,)


In [7]:
# find how many kinds of values in y
y_values = np.unique(model_y)
print(y_values)

[0. 1. 2. 3. 4. 5. 6.]


In [8]:
from cnn_builder2 import *


# learning_rate = 2e-2
# channel = 32


# model_X_normalized = torch.Tensor(model_X)
# model_X_normalized = torch.unsqueeze(model_X_normalized, 1)


# cnn = CNN_Sleeping(channels = channel)

# model = skorch.NeuralNetClassifier(cnn, criterion=torch.nn.CrossEntropyLoss,
#                              device="cuda",
#                              optimizer=torch.optim.SGD ,
#                              lr=learning_rate,
#                              max_epochs=120,
#                              batch_size=128,
#                              callbacks=[skorch.callbacks.EarlyStopping(lower_is_better=True)])

# model.fit(model_X_normalized, np.asarray(model_y, dtype=np.int64))

pred, real_label = train_cnn_model(model_X, model_y)

Start with learning rate: 0.01 and channel: 128
  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        0.9156       0.8214        0.6174  22.6607
      2        0.4751       0.8675        0.4230  22.6434
      3        0.3868       0.8824        0.3580  22.7274
      4        0.3419       0.8904        0.3221  22.9872
      5        0.3161       0.8962        0.3019  23.0845
      6        0.2999       0.8995        0.2886  23.1787
      7        0.2885       0.9028        0.2791  23.2516
      8        0.2800       0.9054        0.2720  24.1647
      9        0.2732       0.9074        0.2665  24.3561
     10        0.2675       0.9090        0.2621  24.6309
     11        0.2626       0.9099        0.2583  23.3717
     12        0.2583       0.9109        0.2551  23.3524
     13        0.2544       0.9119        0.2522  24.1997
     14        0.2508       0.9118        0.2498  23.4349
     15        0.2474   

In [9]:
# distribution of labels y
# print every count of labels
for i in y_values:
    print(i, np.sum(model_y == i))


0.0 281216
1.0 15578
2.0 62754
3.0 5255
4.0 3231
5.0 24275
6.0 1036


In [10]:
from scoring import *

score(pred, real_label)

Class 0: Precision: 0.981, Recall: 0.980
Class 1: Precision: 0.484, Recall: 0.292
Class 2: Precision: 0.798, Recall: 0.914
Class 3: Precision: 0.687, Recall: 0.450
Class 4: Precision: 0.738, Recall: 0.862
Class 5: Precision: 0.746, Recall: 0.726
Class 6: Precision: 0.662, Recall: 0.323
mf1 score: 0.687
acc_score: 0.916
kappa_score: 0.965
